In [1]:
%load_ext tensorboard
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from utils import get_data_generators

In [2]:
TARGET_DIM = 300
BATCH_SIZE = 128

In [3]:
model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(TARGET_DIM, TARGET_DIM, 3),
    include_top=False
)

In [6]:
HP_OPTIMIZER = hp.HParam(
    'optimizer', 
    hp.Discrete([
        'adam', 'sgd'
    ])
)

HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([128, 256, 512]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([64, 128, 256]))

HP_DROPOUT_1 = hp.HParam('dropout_1', hp.Discrete([0.3, 0.4, 0.5]))
HP_DROPOUT_2 = hp.HParam('dropout_2', hp.Discrete([0.2, 0.3, 0.4]))

HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.003, 0.005]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_OPTIMIZER, HP_LEARNING_RATE, HP_NUM_UNITS_1, HP_NUM_UNITS_2, HP_DROPOUT_1, HP_DROPOUT_2],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [7]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True, 
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=30
)

train_generator, validation_generator = get_data_generators(datagen, target_dim=TARGET_DIM, batch_size=BATCH_SIZE)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [10]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        
        opt = None
        global model
        
        preds = model.output 
        preds = tf.keras.layers.GlobalAveragePooling2D()(preds)
        preds = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(preds)
        preds = tf.keras.layers.Dropout(hparams[HP_DROPOUT_1])(preds)
        preds = tf.keras.layers.BatchNormalization()(preds)
        preds = tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer = 'l2')(preds)
        preds = tf.keras.layers.BatchNormalization()(preds)
        preds = tf.keras.layers.Dense(hparams[HP_NUM_UNITS_1], activation=tf.nn.relu)(preds)
        preds = tf.keras.layers.Dropout(hparams[HP_DROPOUT_2])(preds)
        preds = tf.keras.layers.BatchNormalization()(preds)
        preds = tf.keras.layers.Dense(hparams[HP_NUM_UNITS_2], activation=tf.nn.relu, kernel_regularizer ='l2')(preds)
        preds = tf.keras.layers.Dense(11, activation=tf.nn.softmax)(preds)
        
        model = tf.keras.models.Model(model.input, preds)
        
        for layer in model.layers[:-11]:
            layer.trainable = False
        for layer in model.layers[-11:]:
            layer.trainable = True
        
        if hparams[HP_OPTIMIZER] == 'adam':
            opt = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE])
        
        if hparams[HP_OPTIMIZER] == 'sgd':
            opt = tf.keras.optimizers.SGD(learning_rate = hparams[HP_LEARNING_RATE])
        
        model.compile(
            optimizer=opt,
            loss='categorical_crossentropy',
            metrics=['acc']
        )
        
        model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.samples // BATCH_SIZE,
            epochs=1
        )

        _, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // BATCH_SIZE, batch_size=BATCH_SIZE)
        
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0
for optimizer in HP_OPTIMIZER.domain.values:
    
    for rate in HP_LEARNING_RATE.domain.values:
        
        for num_units_1 in HP_NUM_UNITS_1.domain.values:
            
            for num_units_2 in HP_NUM_UNITS_2.domain.values:
                
                for dropout_1 in HP_DROPOUT_1.domain.values:
                    
                    for dropout_2 in HP_DROPOUT_2.domain.values:
                        
                        hparams = {
                            HP_OPTIMIZER: optimizer,
                            HP_LEARNING_RATE: rate,
                            HP_NUM_UNITS_1: num_units_1,
                            HP_NUM_UNITS_2: num_units_2,
                            HP_DROPOUT_1: dropout_1,
                            HP_DROPOUT_2: dropout_2
                        }
                        
                        run_name = "run-%d" % session_num
                        print('--- Starting trial: %s' % run_name)
                        print({h.name: hparams[h] for h in hparams})
                        run('logs/hparam_tuning/' + run_name, hparams)
                        session_num += 1

--- Starting trial: run-0
{'optimizer': 'adam', 'learning_rate': 0.001, 'num_units_1': 128, 'num_units_2': 64, 'dropout_1': 0.3, 'dropout_2': 0.2}
Instructions for updating:
Please use Model.fit, which supports generators.
17/77 [=====>........................] - ETA: 10:12 - loss: 9.1480 - acc: 0.3440